## CELL 1: Setup and Imports

In [101]:
import pandas as pd
from XAI_engine.xai_engine import FakeReviewXAI

print("Imports successful!")

Imports successful!


## CELL 2: Load Dataset

In [102]:
df = pd.read_csv('../data/fake_reviews_dataset.csv')

print(f"Total reviews: {len(df):,}")
print(f"\nLabel distribution:")
print(df['label'].value_counts())
print(f"\nCG = Computer Generated (FAKE)")
print(f"OR = Original (GENUINE)")

Total reviews: 40,432

Label distribution:
label
CG    20216
OR    20216
Name: count, dtype: int64

CG = Computer Generated (FAKE)
OR = Original (GENUINE)


## CELL 3: Prepare Validation Data

In [103]:
n_samples = 100

fake_samples = df[df['label'] == 'CG'].sample(n=n_samples, random_state=42)
genuine_samples = df[df['label'] == 'OR'].sample(n=n_samples, random_state=42)

validation_df = pd.concat([fake_samples, genuine_samples])
validation_df['is_fake'] = validation_df['label'] == 'CG'
validation_df = validation_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Validation set: {len(validation_df)} reviews")
print(f"Fake: {validation_df['is_fake'].sum()}")
print(f"Genuine: {(~validation_df['is_fake']).sum()}")

Validation set: 200 reviews
Fake: 100
Genuine: 100


## CELL 4: Tuning Function
Purpose: Test XAI engine on reviews and record predictions

In [104]:
def tune_thresholds(reviews_df, xai_engine):
    
    """Test XAI engine on labeled data"""
    
    results = []
    
    for idx, row in reviews_df.iterrows():
        try:
            analysis = xai_engine.analyze_review(row['text_'])
            
            predicted_fake = analysis['verdict'] in ['LIKELY FAKE', 'SUSPICIOUS']
            actual_fake = row['is_fake']
            
            results.append({
                'actual_fake': actual_fake,
                'predicted_fake': predicted_fake,
                'verdict': analysis['verdict'],
                'confidence': analysis['confidence'],
                'flag_count': analysis['flag_count'],
                'correct': predicted_fake == actual_fake
            })
        except Exception as e:
            print(f"Error on review {idx}: {e}")
            continue
    
    return results

print("Tuning function ready")

Tuning function ready


## CELL 5: Calculate Metrics function
Purpose: Compute accuracy, precision, recall, F1 score

In [105]:
def calculate_metrics(results):
    
    """Calculate performance metrics"""
    
    tp = sum(1 for r in results if r['actual_fake'] and r['predicted_fake'])
    fp = sum(1 for r in results if not r['actual_fake'] and r['predicted_fake'])
    tn = sum(1 for r in results if not r['actual_fake'] and not r['predicted_fake'])
    fn = sum(1 for r in results if r['actual_fake'] and not r['predicted_fake'])
    
    total = len(results)
    accuracy = (tp + tn) / total if total > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'true_positives': tp,
        'false_positives': fp,
        'true_negatives': tn,
        'false_negatives': fn,
        'total_samples': total
    }

print("Metrics function ready")

Metrics function ready


## Cell 6 : define Threshold Configurations

In [106]:
# ========================================
# CELL 6: Define Threshold Configurations
# ========================================
# Based on feature analysis: Fake vs Real averages
# Fake: sentiment=0.624, word_count=55.75, adj_noun=0.649, first_person=0.051, uniqueness=0.767
# Real: sentiment=0.559, word_count=66.73, adj_noun=0.510, first_person=0.045, uniqueness=0.839

threshold_configs = [
    {
        'name': 'Data-Tuned Conservative',
        'SENTIMENT_EXTREME': 0.65,  # Above real avg (0.559), catches strong fake sentiment
        'WORD_COUNT_MIN': 58,      # Below fake avg (55.75), flags short fakes
        'WORD_COUNT_MAX': 200,
        'ADJ_NOUN_RATIO': 0.62,    # Below fake avg (0.649), catches descriptive fakes
        'FIRST_PERSON_RATIO': 0.048,  # Below fake avg (0.051), flags self-referencing fakes
        'CAPS_RATIO_MAX': 0.01,    # Above fake avg (0.002), flags caps in reals
        'EXCESSIVE_PUNCT_MIN': 1,  # Low, since counts are small
        'UNIQUENESS_RATIO_MIN': 0.79,  # Above fake avg (0.767), flags repetitive fakes
        # Tier 3: Advanced
        'FLESCH_MAX': 85,          # Stricter readability threshold
        'DALE_CHALL_MAX': 7.0,     # Stricter vocabulary threshold
        'BIGRAM_REPETITIVENESS_MIN': 0.90,  # Stricter repetition threshold
        'COMMON_FAKE_NGRAMS_MIN': 3,  # Stricter n-gram threshold
        'TFIDF_SIMILARITY_MAX': 0.4  # Stricter similarity threshold
    },
    {
        'name': 'Data-Tuned Baseline',
        'SENTIMENT_EXTREME': 0.60,  # Midpoint, balances fake/real
        'WORD_COUNT_MIN': 60,      # Between fake (55.75) and real (66.73)
        'WORD_COUNT_MAX': 200,
        'ADJ_NOUN_RATIO': 0.58,    # Midpoint between 0.510 and 0.649
        'FIRST_PERSON_RATIO': 0.048,
        'CAPS_RATIO_MAX': 0.01,
        'EXCESSIVE_PUNCT_MIN': 1,
        'UNIQUENESS_RATIO_MIN': 0.80,  # Closer to real avg (0.839)
        # Tier 3: Advanced
        'FLESCH_MAX': 90,
        'DALE_CHALL_MAX': 8.0,
        'BIGRAM_REPETITIVENESS_MIN': 0.95,
        'COMMON_FAKE_NGRAMS_MIN': 2,
        'TFIDF_SIMILARITY_MAX': 0.5
    },
    {
        'name': 'Data-Tuned Relaxed',
        'SENTIMENT_EXTREME': 0.55,  # Below real avg, more flags
        'WORD_COUNT_MIN': 62,      # Higher, fewer short flags
        'WORD_COUNT_MAX': 200,
        'ADJ_NOUN_RATIO': 0.54,    # Lower, more descriptive flags
        'FIRST_PERSON_RATIO': 0.046,  # Lower, more self-ref flags
        'CAPS_RATIO_MAX': 0.005,   # Lower, more caps flags
        'EXCESSIVE_PUNCT_MIN': 0,  # Very relaxed
        'UNIQUENESS_RATIO_MIN': 0.82,  # Higher, fewer redundancy flags
        # Tier 3: Advanced
        'FLESCH_MAX': 95,          # More relaxed readability
        'DALE_CHALL_MAX': 9.0,     # More relaxed vocabulary
        'BIGRAM_REPETITIVENESS_MIN': 0.98,  # More relaxed repetition
        'COMMON_FAKE_NGRAMS_MIN': 1,  # More relaxed n-gram
        'TFIDF_SIMILARITY_MAX': 0.6  # More relaxed similarity
    }
]

print("Testing 3 data-tuned configurations:")
for config in threshold_configs:
    print(f"  - {config['name']}")

Testing 3 data-tuned configurations:
  - Data-Tuned Conservative
  - Data-Tuned Baseline
  - Data-Tuned Relaxed


## Cell 7: Test For the Best configuration 

In [107]:
# ========================================
# CELL 7: Test All Configurations
# ========================================
all_results = []

for config in threshold_configs:
    print(f"\n{'='*60}")
    print(f"Testing: {config['name']}")
    print(f"{'='*60}")
    
    # Create XAI engine with these thresholds
    xai = FakeReviewXAI()
    
    # Update thresholds (copy config, remove 'name')
    thresholds = {k: v for k, v in config.items() if k != 'name'}
    xai.thresholds.update(thresholds)
    
    # Run tuning
    results = tune_thresholds(validation_df, xai)
    metrics = calculate_metrics(results)
    
    # Store
    config_result = config.copy()
    config_result.update(metrics)
    all_results.append(config_result)
    
    # Print results
    print(f"Accuracy:  {metrics['accuracy']:.2%}")
    print(f"Precision: {metrics['precision']:.2%}")
    print(f"Recall:    {metrics['recall']:.2%}")
    print(f"F1 Score:  {metrics['f1_score']:.2%}")
    print(f"TP: {metrics['true_positives']}, FP: {metrics['false_positives']}")
    print(f"TN: {metrics['true_negatives']}, FN: {metrics['false_negatives']}")

print("\nAll configurations tested!")


Testing: Data-Tuned Conservative
Accuracy:  50.50%
Precision: 50.28%
Recall:    91.00%
F1 Score:  64.77%
TP: 91, FP: 90
TN: 10, FN: 9

Testing: Data-Tuned Baseline
Accuracy:  50.00%
Precision: 50.00%
Recall:    93.00%
F1 Score:  65.03%
TP: 93, FP: 93
TN: 7, FN: 7

Testing: Data-Tuned Relaxed
Accuracy:  50.50%
Precision: 50.25%
Recall:    100.00%
F1 Score:  66.89%
TP: 100, FP: 99
TN: 1, FN: 0

All configurations tested!


## CELL 8: Save Results to threshold_tuning.csv

In [108]:
# ========================================
# CELL 8: Save Results to CSV
# ========================================
results_df = pd.DataFrame(all_results)

column_order = [
    'name', 'SENTIMENT_EXTREME', 'WORD_COUNT_MIN', 'WORD_COUNT_MAX',
    'ADJ_NOUN_RATIO', 'FIRST_PERSON_RATIO',
    'CAPS_RATIO_MAX', 'EXCESSIVE_PUNCT_MIN', 'UNIQUENESS_RATIO_MIN',
    'FLESCH_MAX', 'DALE_CHALL_MAX', 'BIGRAM_REPETITIVENESS_MIN',
    'COMMON_FAKE_NGRAMS_MIN', 'TFIDF_SIMILARITY_MAX',
    'accuracy', 'precision', 'recall', 'f1_score',
    'true_positives', 'false_positives', 'true_negatives', 'false_negatives',
    'total_samples'
]

results_df = results_df[column_order]
results_df.to_csv('../data/threshold_tuning.csv', index=False)

print("Results saved to: data/threshold_tuning.csv")
print("\nResults:")
print(results_df[['name', 'accuracy', 'precision', 'recall', 'f1_score']])

Results saved to: data/threshold_tuning.csv

Results:
                      name  accuracy  precision  recall  f1_score
0  Data-Tuned Conservative     0.505   0.502762    0.91  0.647687
1      Data-Tuned Baseline     0.500   0.500000    0.93  0.650350
2       Data-Tuned Relaxed     0.505   0.502513    1.00  0.668896


## CELL 9: Display Best Configuration

In [109]:
# ========================================
# CELL 9: Find Best Configuration
# ========================================
best_idx = results_df['accuracy'].idxmax()
best_config = results_df.iloc[best_idx]

print("="*60)
print("BEST CONFIGURATION (By Accuracy)")
print("="*60)
print(f"\nName: {best_config['name']}")
print(f"\nPerformance:")
print(f"  Accuracy:  {best_config['accuracy']:.2%}")
print(f"  Precision: {best_config['precision']:.2%}")
print(f"  Recall:    {best_config['recall']:.2%}")
print(f"  F1 Score:  {best_config['f1_score']:.2%}")
print(f"\nThresholds:")
print(f"  SENTIMENT_EXTREME:      {best_config['SENTIMENT_EXTREME']}")
print(f"  WORD_COUNT_MIN:         {best_config['WORD_COUNT_MIN']}")
print(f"  WORD_COUNT_MAX:         {best_config['WORD_COUNT_MAX']}")
print(f"  ADJ_NOUN_RATIO:         {best_config['ADJ_NOUN_RATIO']}")

print(f"  FIRST_PERSON_RATIO:     {best_config['FIRST_PERSON_RATIO']}")
print(f"  CAPS_RATIO_MAX:         {best_config['CAPS_RATIO_MAX']}")
print(f"  EXCESSIVE_PUNCT_MIN:    {best_config['EXCESSIVE_PUNCT_MIN']}")
print(f"  UNIQUENESS_RATIO_MIN:   {best_config['UNIQUENESS_RATIO_MIN']}")

print(f"  FLESCH_MAX:             {best_config.get('FLESCH_MAX')}")
print(f"  DALE_CHALL_MAX:         {best_config.get('DALE_CHALL_MAX')}")
print(f"  BIGRAM_REPETITIVENESS_MIN: {best_config.get('BIGRAM_REPETITIVENESS_MIN')}")
print(f"  COMMON_FAKE_NGRAMS_MIN: {best_config.get('COMMON_FAKE_NGRAMS_MIN')}")
print(f"  TFIDF_SIMILARITY_MAX:   {best_config.get('TFIDF_SIMILARITY_MAX')}")

BEST CONFIGURATION (By Accuracy)

Name: Data-Tuned Conservative

Performance:
  Accuracy:  50.50%
  Precision: 50.28%
  Recall:    91.00%
  F1 Score:  64.77%

Thresholds:
  SENTIMENT_EXTREME:      0.65
  WORD_COUNT_MIN:         58
  WORD_COUNT_MAX:         200
  ADJ_NOUN_RATIO:         0.62
  FIRST_PERSON_RATIO:     0.048
  CAPS_RATIO_MAX:         0.01
  EXCESSIVE_PUNCT_MIN:    1
  UNIQUENESS_RATIO_MIN:   0.79
  FLESCH_MAX:             85
  DALE_CHALL_MAX:         7.0
  BIGRAM_REPETITIVENESS_MIN: 0.9
  COMMON_FAKE_NGRAMS_MIN: 3
  TFIDF_SIMILARITY_MAX:   0.4


## CELL 11: Generate config.py Code
updates config.py with optimal thresholds

In [110]:
# ========================================
# CELL 11: Auto-Update config.py
# ========================================
import os


# Generate the new config content
new_config_content = f'''"""
XAI Engine Configuration
Centralized thresholds for all features (Tier 1 + Tier 2 + Tier 3)
Auto-generated by threshold_tuning.ipynb
"""

# ============================================================================
# ALL THRESHOLDS (TIER 1 + TIER 2 + TIER 3) - OPTIMIZED
# ============================================================================

THRESHOLDS = {{
    # TIER 1: Essential Features
    'SENTIMENT_EXTREME': {best_config['SENTIMENT_EXTREME']},
    'WORD_COUNT_MIN': {int(best_config['WORD_COUNT_MIN'])},
    'WORD_COUNT_MAX': {int(best_config['WORD_COUNT_MAX'])},
    'ADJ_NOUN_RATIO': {best_config['ADJ_NOUN_RATIO']},
    'FIRST_PERSON_RATIO': {best_config['FIRST_PERSON_RATIO']},
    
    # TIER 2: Important Features
    'CAPS_RATIO_MAX': {best_config['CAPS_RATIO_MAX']},
    'EXCESSIVE_PUNCT_MIN': {int(best_config['EXCESSIVE_PUNCT_MIN'])},
    'UNIQUENESS_RATIO_MIN': {best_config['UNIQUENESS_RATIO_MIN']},
    
    # TIER 3: Advanced Features
    'FLESCH_MAX': {best_config['FLESCH_MAX']},  # Flag if readability too high (fake-like simplicity)
    'DALE_CHALL_MAX':{best_config['DALE_CHALL_MAX']},  # Flag if too simple vocabulary
    'BIGRAM_REPETITIVENESS_MIN': {best_config['BIGRAM_REPETITIVENESS_MIN']},  # Flag if too repetitive
    'COMMON_FAKE_NGRAMS_MIN': {best_config['COMMON_FAKE_NGRAMS_MIN']},  # Flag if >=2 common fake phrases
    'TFIDF_SIMILARITY_MAX': {best_config['TFIDF_SIMILARITY_MAX']}  # Flag if too similar to generic reviews
}}


def load_config():
    """Return a copy of thresholds"""
    return THRESHOLDS.copy()


def update_thresholds(new_thresholds):
    """Update thresholds (used by threshold tuning)"""
    THRESHOLDS.update(new_thresholds)
'''

# Write to config.py
config_path = '../xai_engine/config.py'

try:
    with open(config_path, 'w') as f:
        f.write(new_config_content)
    
    print("config.py successfully updated!")
    print(f"\nFile location: {os.path.abspath(config_path)}")
    
except Exception as e:
    print(f"Error updating config.py: {e}")
    print("\nManual update required. Copy this code:\n")
    print(new_config_content)

config.py successfully updated!

File location: c:\Users\Amr\Documents\GitHub\Team-C-repository\xai_engine\config.py
